<a href="https://colab.research.google.com/github/Vashistsanskar/Myfirstproject/blob/main/Logistic_Reg_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import pickle
from sklearn.base import BaseEstimator,TransformerMixin

In [5]:
class Adultfemale_agebins(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.some=None

  def fit(self,X_train=None,y_train=None):
    return self

  def transform(self,X_train,y_train=None):
    #female_adult_col_feature
    X_train = pd.DataFrame(X_train,columns=['Fare','Pclass','Sex','Age','Embarked','Parch'])
    X_train = X_train.copy()
    mask = (X_train['Sex']=='female') & (X_train['Age']>18) & (X_train['Age']<39)
    X_train['Adult_female'] = np.where(mask,1,0)

    #age_bins_feature
    bins = [0, 15, 28, 39, 60, 80]  # Bin edges
    labels = [4.0, 1.0, 2.0, 3.0, 0.0]  # Ordinal labels as specified

    X_train['AgeGroup'] = pd.cut(X_train['Age'], bins=bins, labels=labels,include_lowest=True)
    X_train.drop(columns='Age',inplace=True)

    return X_train

In [6]:
class Ageimpute(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.mean_age = None

  def fit(self,X_train,y_train=None):
    self.mean_age = X_train.groupby(['Pclass','Sex'])['Age'].mean()
    return self

  def transform(self,X_train,y_train=None):
        X_train = X_train.copy()
        #age_impute
        for (pclass,sex),mean_age in self.mean_age.items():
            mask = (X_train['Age'].isnull()) & (X_train['Pclass'] == pclass) & (X_train['Sex']==sex)
            X_train.loc[mask, 'Age'] = mean_age
        return X_train

In [7]:
class pclass_sex__embarked_encode(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.one=None

  def fit(self,X_train=None,y_train=None):
    return self

  def transform(self,X_train,y_train=None):
    #pclass encode
    X_train = pd.DataFrame(X_train,columns=['Embarked','Fare','Pclass','Sex','parch','Adult_female','Age_group'])
    X_train = X_train.copy()
    X_train['Pclass'].replace({
    1:3.0,
    3:1.0
    },inplace=True)
    #sex encode
    X_train['Sex'].replace({
      'female':1.0,
      'male':0.0
    },inplace=True)

    #embarked_encode
    X_train['Embarked'].replace({
            'S':0.0,
            'Q':1.0,
            'C':2.0
        },inplace=True)

    return X_train

In [8]:
class dropT(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.drop=None

  def fit(self,X_train=None,y_train=None):
    return self

  def transform(self,X_train,y_train=None):
    X_train = X_train.copy()

    X_train.drop(columns=['PassengerId','Ticket','Cabin','Name','SibSp'],inplace=True)

    return X_train

In [9]:
PIPE = pickle.load(open('/content/pipe_grid.pkl','rb'))

In [10]:
test_df = pd.read_csv('/content/test.csv')

In [11]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
test_df.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [15]:
Prediction = PIPE.predict(test_df)

/tmp/ipython-input-7-1480020728.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Pclass'].replace({
/tmp/ipython-input-7-1480020728.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train['Pclass'].replace({
/tmp/ipython-input-7-1480020728.py:17: FutureWarning: A value is trying to

In [16]:
Prediction

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,